# Exercice 1. CNN for object classification

Le jeu de données Toy Dataset - v7 2023-04-19 1:21am est fourni par Roboflow (https://universe.roboflow.com/tips-wis6y/toy-dataset-bfq0d). Vous en trouverez une copie sur la page Moodle.

Les données brutes se trouvent dans le sous-dossier « images ». Les données d'étiquetage se trouvent dans le sous-dossier « labels ».

Le format des données d'étiquetage est le suivant :

    categ, cx, cy, w, h

où categ représente le numéro de catégorie, cx et cy les coordonnées du centre de l'objet (sur une échelle de 0 à 1), et w et h ses valeurs horizontale et verticale (exprimées en pourcentage de la largeur et de la hauteur). (cx, cy, w, h) constitue le cadre englobant de l'objet. Cette représentation est conforme au format YOLO8.

Afin d'exploiter tous les objets présents dans les images, lors du chargement du jeu de données, nous pouvons extraire les images des objets à partir des images originales en les recadrant à l'intérieur de leur cadre englobant.

Deux configurations sont présentées ci-dessous :

    all objects versus all objects (load_objects)
    one object versus all (load_objects_as_binary_problem)

**Q1** Vous pouvez utiliser le code tel quel, mais vous pouvez également le modifier afin d'obtenir un équilibre entre les classes, notamment pour la classification binaire. Pourquoi pensez-vous que cela peut être intéressant d'équilibrer entre les classes ?

__Réponse__:

In [39]:
import os
import csv
import cv2
import numpy as np
import math

train_path="../toy-dataset/train/"
test_path="../toy-dataset/test/"

def load_objects(imgs_path,max_samples=np.iinfo(np.int64)):
    y_obj=[]
    x_obj=[]
    imgs_files = os.listdir( imgs_path+"images/" )
    for i,img_file in enumerate(imgs_files):
        if (len(x_obj)==max_samples):
            break
        label_file=img_file[:-4]+".txt"
        img_init=cv2.imread(imgs_path+"images/"+img_file)
        labels=csv.reader(open(imgs_path+"labels/"+label_file,"r"),delimiter=' ')
        rows = list(labels)
        if len(rows)>0:
            for j,row in enumerate(rows):
                if (len(x_obj)==max_samples):
                    break
                y_obj.append(row[0])
                bbox=np.array(row[1:],dtype=np.float32) if row[0]==1 else np.array(row[1:],dtype=np.float32)
                w=int(bbox[2]*img_init.shape[0]*2)
                h=int(bbox[3]*img_init.shape[1]*2)
                x0=max(0,int(math.trunc(np.float64(bbox[0]*img_init.shape[0]-w/2))))
                y0=max(0,int(math.trunc(np.float64(bbox[1]*img_init.shape[1]-h/2))))
                x1=min(img_init.shape[0],int(math.trunc(np.float64(x0+w))))
                y1=min(img_init.shape[1],int(math.trunc(np.float64(y0+h))))
                x_obj.append(np.copy(img_init[y0:y1,x0:x1]))
    return x_obj,y_obj
                
def load_objects_as_binary_problem(imgs_path,max_samples=np.iinfo(np.int32)):
    y_obj=[]
    x_obj=[]
    imgs_files = os.listdir( imgs_path+"images/" )
    for i,img_file in enumerate(imgs_files):
        if (len(x_obj)==max_samples):
            break
        label_file=img_file[:-4]+".txt"
        #print(i,img_file,label_file)
        img_init=cv2.imread(imgs_path+"images/"+img_file)
        labels=csv.reader(open(imgs_path+"labels/"+label_file,"r"),delimiter=' ')
        rows = list(labels)
        if len(rows)>0:
            for j,row in enumerate(rows):
                if (len(x_obj)==max_samples):
                    break
                if (row[0]=='4'):
                    y_obj.append(int(row[0]))
                else:
                    y_obj.append(0)
                bbox=np.array(row[1:],dtype=np.float32) if row[0]==1 else np.array(row[1:],dtype=np.float32)
                w=int(bbox[2]*img_init.shape[0])
                h=int(bbox[3]*img_init.shape[1])
                x0=max(0,int(math.trunc(np.float64(bbox[0]*img_init.shape[0]-w/2))))
                y0=max(0,int(math.trunc(np.float64(bbox[1]*img_init.shape[1]-h/2))))
                x1=min(img_init.shape[0],int(math.trunc(np.float64(x0+w))))
                y1=min(img_init.shape[1],int(math.trunc(np.float64(y0+h))))
                x_obj.append(img_init[y0:y1,x0:x1].copy())
    return x_obj,y_obj

def load_objects_as_binary_problem_balanced(imgs_path,max_positivesamples=np.iinfo(np.int32)):
    return [],[]

## Architectures légères

Essayez des architectures simples combinant une ou deux couches de convolution et de pooling, suivies d'une ou deux couches denses.

Pour la classification binaire, la dernière couche ne comporte qu'un seul neurone.

Pour la classification multicatégorielle, la dernière couche comporte autant de neurones que de catégories. Le neurone gagnant est celui qui a la valeur de sortie la plus élevée. Un encodage des étiquettes catégorielles est nécessaire pour préparer les données d'entraînement (voir `tf.keras.utils.to_categorical`).

Vous pouvez choisir le framework (PyTorch ou TensorFlow) avec lequel vous êtes à l'aise.

Vous pouvez travailler en local ou sur Google Colab.

Vous trouverez ci-dessous un exemple de code TensorFlow incomplet pour résoudre ce problème, mais vous pouvez le remplacer complétement par une solution de votre choix.

In [ ]:
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import math
import sklearn.cluster as skc
import sklearn.svm as svm

# multi classification 

x_train, y_train = load_objects(train_path, 1000)


# Choose a target size
TARGET_SIZE = (128, 128)

def resize_images(images, target_size):
    return [tf.image.resize(img, target_size) for img in images]

x_train_resized = resize_images(x_train, TARGET_SIZE)
x_train_tensor = tf.stack(x_train_resized)

# preparing the network input tensor

inputs = tf.keras.Input(shape=(*TARGET_SIZE, x_train_tensor.shape[-1]))

# Convolutional Layer #1
conv1 = tf.keras.layers.Conv2D(
    filters=64,
    kernel_size=[5, 5],
    padding="valid",
    activation=tf.nn.relu)(inputs)

# Pooling Layer
pool1 = tf.keras.layers.MaxPooling2D(pool_size=[2, 2], strides=2)(conv1)

# Flatten the pooling output
flat = tf.keras.layers.Flatten()(pool1)

# Dense layer
mlp = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)(flat)

# Output layer - number of units should match your number of classes
y_train = [int(label) for label in y_train]
y_train = np.array(y_train)

# Get number of classes
num_classes = len(set(y_train))
output_classif = tf.keras.layers.Dense(units=num_classes, activation='softmax')(mlp)

model = tf.keras.Model(inputs=inputs, outputs=output_classif, )

sparsecatloss = tf.keras.losses.SparseCategoricalCrossentropy()

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=sparsecatloss,  # Use categorical cross-entropy for multi-class classification
    metrics=['accuracy']
)

model.summary()

### binary classification 

# Load data
x_train_bin, y_train_bin = load_objects_as_binary_problem(train_path, 1000)

# Convert labels to integers (0 or 1)
y_train_bin = np.array([int(label) for label in y_train_bin])

# Resize images to fixed size
x_train_bin_resized = [tf.image.resize(img, TARGET_SIZE) for img in x_train_bin]
x_train_bin_tensor = tf.stack(x_train_bin_resized)

# Build the model
inputs_bin = tf.keras.Input(shape=(*TARGET_SIZE, x_train_bin_tensor.shape[-1]))

conv1_bin = tf.keras.layers.Conv2D(
    filters=64,
    kernel_size=[5, 5],
    padding="valid",
    activation=tf.nn.relu)(inputs_bin)

pool1_bin = tf.keras.layers.MaxPooling2D(pool_size=[2, 2], strides=2)(conv1_bin)

flat_bin = tf.keras.layers.Flatten()(pool1_bin)

mlp_bin = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)(flat_bin)

output_bin = tf.keras.layers.Dense(units=1, activation='sigmoid')(mlp_bin)

model_bin = tf.keras.Model(inputs=inputs_bin, outputs=output_bin)

model_bin.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

model_bin.summary()


Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_14 (InputLayer)     │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 124, 124, 64)   │         4,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_14 (Flatten)            │ (None, 246016)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 256)            │    62,980,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 14)             │         3,598 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 62,988,814 (240.28 MB)

 Trainable params: 62,988,814 (240.28 MB)

 Non-trainable params: 0 (0.00 B)

Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_15 (InputLayer)     │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 124, 124, 64)   │         4,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_15 (Flatten)            │ (None, 246016)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 256)            │    62,980,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 62,985,473 (240.27 MB)

 Trainable params: 62,985,473 (240.27 MB)

 Non-trainable params: 0 (0.00 B)

In [44]:
x_train_resized = resize_images(x_train, TARGET_SIZE)
x_train_tensor = tf.stack(x_train_resized)

y_train = [int(label) for label in y_train]
y_train = np.array(y_train)

model.fit(x=x_train_tensor, y=y_train, batch_size=32, epochs=5)



Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 126ms/step - accuracy: 0.2500 - loss: 322.4623
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 124ms/step - accuracy: 0.6250 - loss: 12.4636
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 124ms/step - accuracy: 0.8820 - loss: 1.0278
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 123ms/step - accuracy: 0.9400 - loss: 0.3829
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 123ms/step - accuracy: 0.9670 - loss: 0.2838


In [46]:
x_train_bin, y_train_bin = load_objects_as_binary_problem(train_path, 1000)

x_train_bin_resized = [tf.image.resize(img, TARGET_SIZE) for img in x_train_bin]
x_train_bin_tensor = tf.stack(x_train_bin_resized)
y_train_bin = np.array([int(label) for label in y_train_bin])

model_bin.fit(x_train_bin_tensor, y_train_bin, epochs=5, batch_size=32)

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 129ms/step - accuracy: 0.8680 - loss: 38.9859
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 125ms/step - accuracy: 0.8230 - loss: 0.5685
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 125ms/step - accuracy: 0.9750 - loss: 0.3414
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 124ms/step - accuracy: 0.9760 - loss: 0.1502
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 125ms/step - accuracy: 0.9580 - loss: 0.0958


In [48]:
model.save('model_multiclass.keras')
model_bin.save('model_binary.keras')

**Q2** Évaluez le modèle sur les ensembles d'entraînement et de test pour les deux modalités de classification (binaire et multiclasse).
Précisez les performances en train et test pour les différentes configurations essayées. 

__Réponse__:


**Q3** Que se passe-t-il si vous changez de modalité de classification ? Faut-il réentraîner entièrement le modèle ? Quelles parties peuvent être conservées ?

__Réponse__ : Les parties de convolutions peuvent êtres conservées, la seule couche qu'il faut changer est la couche de MLP qui fait la classification 

**Q4** Avez-vous des remarques concernant la classification d'objets par BoW (TP2) par rapport à la classification d'objets par CNN ?

__Réponse__ : C'est plus efficace

In [35]:
### Q3 Model evaluation multi classification 

# Load and preprocess test data
x_test, y_test = load_objects(test_path, 1000)

# Convert labels to integers
y_test = np.array([int(label) for label in y_test])

# Resize images (same as training)
x_test_resized = [tf.image.resize(img, TARGET_SIZE) for img in x_test]
x_test_tensor = tf.stack(x_test_resized)

# Evaluate the model
loss, accuracy = model.evaluate(x_test_tensor, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")



26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.8036 - loss: 2.0518
Test Loss: 2.0518
Test Accuracy: 0.8036


In [47]:
x_test_bin, y_test_bin = load_objects_as_binary_problem(test_path, 1000)
y_test_bin = np.array([int(label) for label in y_test_bin])
x_test_bin_resized = [tf.image.resize(img, TARGET_SIZE) for img in x_test_bin]
x_test_bin_tensor = tf.stack(x_test_bin_resized)

# Evaluate
loss_bin, accuracy_bin = model_bin.evaluate(x_test_bin_tensor, y_test_bin)

# Predictions
y_pred_bin_probs = model_bin.predict(x_test_bin_tensor)
y_pred_bin = (y_pred_bin_probs > 0.5).astype(int).flatten()

# Metrics
from sklearn.metrics import classification_report
print(classification_report(y_test_bin, y_pred_bin))

26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.9418 - loss: 0.0645
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step
              precision    recall  f1-score   support

           0       0.99      0.98      0.98       795
           1       0.00      0.00      0.00         0
           4       0.00      0.00      0.00        30

    accuracy                           0.94       825
   macro avg       0.33      0.33      0.33       825
weighted avg       0.95      0.94      0.95       825



/Users/ainazeaze/etude/amvo/amvo/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/ainazeaze/etude/amvo/amvo/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/ainazeaze/etude/amvo/amvo/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metr

## Transfer learning (optionnel et pas nécessaire pour la suite)

Dans cette partie, nous allons refaire le même exercice en réutilisant un modèle existant, tel que MobileNet, qui a déjà été entraîné.

Une vaste collection de modèles pré-entraînés est disponible dans des frameworks comme TensorFlow.

Le code ci-dessous utilise MobileNet, mais vous pouvez facilement changer de modèle en instanciant celui de votre choix.

Pour cette partie, il est recommandé d'utiliser Google Colab ou Grid5K.
Dans ce cas, vous devrez également envisager de télécharger les données sur votre Drive ou votre répertoire personnel Grid5K.

In [ ]:
#load and prepare the data
#the x_train should be ideally reshaped to (_,224,224,3)

#from tensorflow.keras import layers, models
#from tensorflow.keras.applications.mobilenet import MobileNet


#base_model=MobileNet(weights="imagenet", include_top=False, input_shape=x_train[0].shape)
#include_top is False in order to drop the default Dense Layers and to be able to add your own
#base_model.trainable = False ## Not trainable weights

#flat=layers.Flatten()(base_model)
#mlp=tf.keras.layers.Dense(units=256, activation=tf.nn.relu)(flatten_layer)
#...
#output_classif = ...(mlp)

#model = tf.keras.Model(inputs=inputs, outputs=output_classif)

#catloss=tf.keras.losses.CategoricalCrossentropy()
#binloss=tf.keras.losses.BinaryCrossentropy()

#model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
#              loss=...,metrics=['accuracy'])

#model.summary()

**Q5** Évaluez le modèle sur les ensembles d'entraînement et de test pour les deux modalités de classification (binaire et multiclasse).
Précisez les performances en train et test pour les différentes configurations essayées. 

__Réponse__:


**Q6** Que se passe-t-il si vous changez de modalité de classification ? Faut-il réentraîner entièrement le modèle ? Quelles parties peuvent être conservées ?

__Réponse__ :

**Q7** Avez-vous des remarques concernant la classification d'objets par BoW (TP2) par rapport à la classification d'objets par CNN ?

__Réponse__ :